# <span style="background-color:#fff5b1"><span style="color:black"><B>보험사 마케팅 활용을 위한 고객 군집화 모델링

## **Mission 2. 데이터 전처리**

### [미션 이해하기]
* 여러분들은 K 보험사의 직원입니다. K 보험사는 효과적인 마케팅 전략을 완성시키기 위해 데이터 기반 마케팅을 시작했습니다.
  <br>고객의 행동,성향, 니즈 등의 고객 데이터를 마케팅 전략에 반영하여 고객별로 적절한 마케팅 Action을 주는 미션을 받았습니다.
* 고객별로 특징을 적절하게 반영할 수 있는 마케팅 방안은 고객을 segment 하는 것 인데, 어떤 방법으로 고객을 묶을 수 있을까요?
  <br>바로, <font color="1E90FF"><b>군집분석을 기반으로 한 고객 segment를 개발 하는 것입니다.</font>


<b>[1일차] mission 2 목표<b>: 군집분석을 기반으로 하는 고객 segment 적용 전 주요 데이터 전처리를 수행해보기

---------------------------------------------------------------------------------------------
# **0. 환경설정(경로설정)**
* 세부 요구사항
    - 경로 설정 : 구글 colab
        * 구글 드라이브 바로 밑에 dxproject 폴더를 만들고,
        * 데이터 파일을 복사해 넣습니다.

In [1]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Path
path = '/content/drive/MyDrive/dxproject/'

---------------------------------------------------------------------------------------------
#### <b>[실습프로세스]<b>

1. 범주 인코딩
    * 각 범주별 0, 1로 인코딩 하기
2. 스케일링


---------------------------------------------------------------------------------------------
# **1. 범주 인코딩**
* 범주형 변수 값(텍스트) 을 수치형 값(숫자)으로 변환하는 방법을 사용하여 알고리즘이 인식할 수 있도록 변환해 봅시다.
* 범주 인코딩 방법은 Label-encoding 과 one-hot-encoding 등이 있습니다.
    * Label-encoding : 0, 1, 2, 3, ...
    * one-hot-encoding : 가변수화
* 우리는 어떻게 범주 인코딩을 할 수 있을까요?

## **(1) 데이터 불러오기**


In [79]:
#[문제 1] 필요 라이브러리 로딩
# numpy, pandas, matplotlib, seaborn, os 를 임포트 하기

import numpy as np
import pandas as pd
import seaborn as sns
import os

In [80]:
# 한글 세팅

# 1. 나눔고딕 폰트 설치
!apt-get -qq update
!apt-get -qq install -y fonts-nanum

# 2. 런타임에 폰트 캐시 갱신
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import matplotlib as mpl

font_dirs = ['/usr/share/fonts/truetype/nanum']
font_files = fm.findSystemFonts(fontpaths=font_dirs)

for font_file in font_files:
    fm.fontManager.addfont(font_file)

# 3. 폰트 이름 확인 후 설정
nanum_font_name = fm.FontProperties(fname=font_files[0]).get_name()
print(f"설정된 폰트 이름: {nanum_font_name}")

# matplotlib에 폰트 반영
mpl.rc('font', family=nanum_font_name)
mpl.rcParams['axes.unicode_minus'] = False

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
설정된 폰트 이름: NanumGothic


In [81]:
# [문제 2] 데이터 불러오기
# data 변수에 'customers_seg.csv' 파일을 불러오기

data = pd.read_csv(path +'customers_seg.csv')


In [82]:
# [문제 3] 데이터 상위 5개 조회

data.head()

,CID,AGE,등록(계/피),고용상태,성별,Willingness to pay/Stay,갱신,상품타입,교육수준,소득,...,결혼여부,월 납입액,VOC,타 상품 보유 현황,온라인방문빈도,갱신인센티브,판매채널,총지불금액,자동차,거주지사이즈
0,1,3,피보험자,무직,1,2.616381,0,기본,대학졸업,0,...,미혼,80,0,1,자주방문,없음,자사영업,631000,일반세단,소
1,2,1,계약자,고용,0,6.352530,0,기본,고졸이하,102887400,...,미혼,80,1,4이상,비방문,포인트,인터넷,54000,일반세단,중
2,3,2,계약자,휴직,0,4.974354,0,기본,대학졸업,22159500,...,기혼,60,0,4이상,비방문,할인,자사영업,362000,일반세단,중
3,4,5,계약자,고용,1,13.480284,0,고급,고졸이하,51562500,...,기혼,110,0,2,비방문,할인,자사영업,1264000,컴팩트카,대
4,5,4,계약자,고용,0,9.776436,0,기본,석사,26820200,...,기혼,120,0,3,비방문,없음,대리점,947000,SUV,중


## **(2) 활용할 변수 선정하기**

In [83]:
# [문제 4] 데이터 셋의 모든 컬럼명(변수명)을 확인해보자.(columns로 확인)

data.columns

Index(['CID', 'AGE', '등록(계/피)', '고용상태', '성별', 'Willingness to pay/Stay', '갱신',
       '상품타입', '교육수준', '소득', '지역', '결혼여부', '월 납입액', 'VOC', '타 상품 보유 현황',
       '온라인방문빈도', '갱신인센티브', '판매채널', '총지불금액', '자동차', '거주지사이즈'],
      dtype='object')

In [84]:
# [문제 5] mission 1에서 선정한 11개의 변수만 가지고 있는 데이터 프레임 생성하기

# 1. data를 data_choice 변수에 copy 해놓기

data_choice = data

# 2. col 에 선정한 11개 변수를 리스트로 할당해보자.
# [참고] 11개 변수: 'AGE', '고용상태', 'Willingness to pay/Stay', '상품타입',
# '교육수준', '소득', '월 납입액','타 상품 보유 현황', '총지불금액', '거주지사이즈','자동차'

col = ['AGE', '고용상태', 'Willingness to pay/Stay', '상품타입','교육수준', '소득',
       '월 납입액','타 상품 보유 현황', '총지불금액', '거주지사이즈','자동차']

# 3.data_choice 변수에 col 할당 후 data_choice 데이터를 확인해보자.

data_choice = data_choice[col]

data_choice.columns


Index(['AGE', '고용상태', 'Willingness to pay/Stay', '상품타입', '교육수준', '소득', '월 납입액',
       '타 상품 보유 현황', '총지불금액', '거주지사이즈', '자동차'],
      dtype='object')

## **(3) 범주 인코딩**

**[명목형(텍스트) 컬럼 처리하기]**
   * 순서의 의미가 없는 컬럼은 인코딩 시 주의! 변수의 의미를 고려하여 처리가 필요하다.
   * Mission 1에서 변수마다 탐색적 분석을 통해 의미를 찾아본 것을 활용해야한다.
   * 우리는 1 또는 0으로 이산화(두개의 범주로 구분)를 해보려고 한다.

In [85]:
# [문제 6] 범주형 변수가 무엇이 있는지 확인해보자(info 활용)

data_choice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   AGE                      12000 non-null  int64  
 1   고용상태                     12000 non-null  object 
 2   Willingness to pay/Stay  12000 non-null  float64
 3   상품타입                     12000 non-null  object 
 4   교육수준                     12000 non-null  object 
 5   소득                       12000 non-null  int64  
 6   월 납입액                    12000 non-null  int64  
 7   타 상품 보유 현황               12000 non-null  object 
 8   총지불금액                    12000 non-null  int64  
 9   거주지사이즈                   12000 non-null  object 
 10  자동차                      12000 non-null  object 
dtypes: float64(1), int64(4), object(6)
memory usage: 1.0+ MB


**-> 범주형 변수는 '고용상태', '상품타입', '교육수준', '타 상품 보유 현황'. '거주지 사이즈','자동차'로 총 6개가 있다.**

---------------------------

**[Hint]**
- **<b>'고용상태', '상품 타입', '교육수준','거주지 사이즈','자동차'</b>** 는 다중범주로 순서에 의미가 없어 변수의 의미를 고려해서  <font color="1E90FF"><b>0과 1로 이산화로 범주 인코딩</font>
- **'타 상품 보유 현황'** 의 경우 숫자로 되어있는 범주에 <font color="1E90FF"><b>'4이상'의 범주의 '이상'이라는 문자열 제거</font>

---------------

### **1) 고용상태**
<font color="1E90FF"><b>★ 다중범주로 순서에 의미가 없기에 변수의 의미를 고려해서 인코딩 해보자.</font>

* 1-> 직업을 가지고 있으면서 소득이 발생하는 범주
* 0-> 직업이 없거나, 소득이 발생하지 않는 범주
* 특이하게 false 값 도 존재 하기 때문에 확인 필요

In [86]:
# [문제 7] '고용상태' 범주 종류 확인(.unique() 혹은 .value_counts())

data_choice['고용상태'].value_counts()

,count
고용상태,
고용,7518
무직,3028
휴직,1094
FALSE,360


In [88]:
# [같이 하기]'고용상태' 범주를 인코딩 해보기
# np.where 을 이용해서 조건에 따른 값으로 변환하기
# 고용상태가 '고용' 이면 1, 아니면 0

data_choice['고용상태'] = np.where(data_choice['고용상태'] == '고용', 1, 0)

<ipython-input-88-9abebc7d0a87>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_choice['고용상태'] = np.where(data_choice['고용상태'] == '고용', 1, 0)


In [89]:
#[같이 하기] '고용상태'가 변환된 것을 확인해보기

data_choice['고용상태'].head()

,고용상태
0,0
1,0
2,0
3,0
4,0


<b> why? Labelencoding 과 one-hot-encoding을 안하는가?
   > Label encoding을 하면 범주 수에 따라서 0~n 까지 나눠지기 때문에 이진의 의미를 할 수가 없다. (ex.고용상태 기준 --> false 0, 고용1, 무직2, 휴직3)
    <br> 연속적수치값으로 변환하는데 수치적 의미가 차이가 없고 3개 이상 범주는 주의해서 사용해야한다.
    <br> one-hot-encoding은 범주형 변수를 k개 변수로 변환하는데 범주수 만큼 열이 확대가 되어서...컨트롤하고 분석하기가 힘들다
    <br>가끔 이런 단순한 방법이 더 좋을 때도 있다( 간단하게 분석하는 경우...)

### **2) 상품타입**
<font color="1E90FF"><b>★ 다중범주로 순서에 의미가 없기에 변수의 의미를 고려해서 인코딩 해보자.</font>

* 1-> 약 9%를 가지고 있는 고급형
* 0-> 약 91%를 가지고 있는 기본, 중급형
* 고급형을 가지고 있는 집단과, 그 외 집단으로 분류

In [90]:
# [문제 8] '상품타입' 범주와 비율 확인

data_choice['상품타입'].value_counts(normalize = True)

,proportion
상품타입,
기본,0.606500
중급,0.303917
고급,0.089583


In [91]:
# [문제 9]'상품타입'범주를 인코딩 해보기
# '상품타입'이 '고급' 1, 아니면 0

data_choice['상품타입'] = np.where(data_choice['상품타입'] == '고급', 1, 0)

<ipython-input-91-cf699c62a85f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_choice['상품타입'] = np.where(data_choice['상품타입'] == '고급', 1, 0)


In [92]:
#[문제 10] '상품타입'이 변환된 것을 확인해보기

data_choice['상품타입'].head()

,상품타입
0,0
1,0
2,0
3,1
4,0


### **3) 교육수준**
<font color="1E90FF"><b>★ 다중범주로 순서에 의미가 없기에 변수의 의미를 고려해서 인코딩 해보자.</font>
* 1-> 약 12%를 가지고 있는 석/박사
* 0-> 약 88%의 비중을 가지고 있는 고졸이하, 대학졸업자
* 교육수준이 높은 집단과, 그 외 집단으로 분류할 수 있을 것 같음

In [93]:
#[문제 11] '교육수준' 범주와 비율 확인

data_choice['교육수준'].value_counts(normalize = True)

,proportion
교육수준,
대학졸업,0.592083
고졸이하,0.289750
석사,0.081583
박사,0.036583


In [94]:
#[문제 12]'교육수준' 범주를 인코딩 해보기
# '교육수준'이 '석사' 혹은 '박사'이면 1, 아니면 0

data_choice['교육수준'] = np.where((data_choice['교육수준'] == '석사') | (data_choice['교육수준'] == '박사') , 1, 0)


<ipython-input-94-8154bf81253e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_choice['교육수준'] = np.where((data_choice['교육수준'] == '석사') | (data_choice['교육수준'] == '박사') , 1, 0)


In [95]:
#[문제 13] '교육수준'이 변환된 것을 확인해보기

data_choice['교육수준'].head()

,교육수준
0,0
1,0
2,0
3,0
4,1


### **4) 타 상품 보유 현황**
<font color="1E90FF"><b>★ 범주의 1-4까지가 의미를 가지는 범주를 인코딩 해보자.</font>
* 4 이상의 타상품 보유하는 경우 유의미한 데이터로 가질 수 있어 순서의 의미를 살려서 인코딩 진행

In [96]:
#[문제 14] '타 상품 보유 현황' 범주와 비율 확인

data_choice['타 상품 보유 현황'].value_counts(normalize = True)

,proportion
타 상품 보유 현황,
1,0.356917
4이상,0.262917
2,0.251417
3,0.128750


In [97]:
# [문제 15] 문자열 제거 하기
# '타 상품 보유 현황'은 숫자형 이지만, 4이상의 '이상'이라는 문자열을 제거해서 정리가 필요
# np.where를 이용해서 '이상'을 제거해 봅시다.
data_choice['타 상품 보유 현황'] = np.where(
    data_choice['타 상품 보유 현황'].astype(str).str.contains('이상'),
    data_choice['타 상품 보유 현황'].str.replace('이상', '', regex=False),
    data_choice['타 상품 보유 현황']
)

# 문자열 제거 후 숫자로 변환
data_choice['타 상품 보유 현황'] = pd.to_numeric(data_choice['타 상품 보유 현황'], errors='coerce')


<ipython-input-97-28959a46cfb3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_choice['타 상품 보유 현황'] = np.where(
<ipython-input-97-28959a46cfb3>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_choice['타 상품 보유 현황'] = pd.to_numeric(data_choice['타 상품 보유 현황'], errors='coerce')


In [98]:
#[문제 16]'타 상품 보유 현황'이 변환된 것을 확인해보기
data_choice['타 상품 보유 현황'].value_counts(normalize = True)


,proportion
타 상품 보유 현황,
1,0.356917
4,0.262917
2,0.251417
3,0.128750


### **5) 거주지 사이즈**
<font color="1E90FF"><b>★ 다중범주로 순서에 의미가 없기에 변수의 의미를 고려해서 인코딩 해보자.</font>
* 1-> 약 10%를 가지고 있는 거주지 대
* 0-> 약 90%를 가지고 있는 중,소
* 거주지의 사이즈가 대로 가지고 있는 집단과, 그 외 집단으로 분류할 수 있을 것 같음

In [99]:
#[문제 17] '거주지사이즈' 범주와 비율 확인

data_choice['거주지사이즈'].value_counts(normalize = True)

,proportion
거주지사이즈,
중,0.701917
소,0.194333
대,0.103750


In [100]:
#[문제 18]'거주지사이즈' 범주를 인코딩 해보기
# '대'를 1, 나머지를 0

data_choice['거주지사이즈'] = np.where(data_choice['거주지사이즈'] == '대' , 1, 0)

<ipython-input-100-9115d645a76b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_choice['거주지사이즈'] = np.where(data_choice['거주지사이즈'] == '대' , 1, 0)


In [101]:
#[문제 19]'거주지사이즈'가 변환된 것을 확인해보기
data_choice['거주지사이즈'].value_counts(normalize = True)


,proportion
거주지사이즈,
0,0.89625
1,0.10375


### **6) 자동차**
<font color="1E90FF"><b>★ 다중범주로 순서에 의미가 없기에 변수의 의미를 고려해서 인코딩 해보자.</font>
* 1-> 약 9%를 가지고 있는 고급차/스포츠카
* 0-> 약 91%를 가지고 있는 그외(일반세단, 컴팩트카,suv)
* 고급형(고급차.스포츠카)와 그 외의 집단으로 분류할 수 있을 것 같음(소득에 연관)

In [102]:
#[문제 20] '자동차' 범주와 비율 확인

data_choice['자동차'].value_counts(normalize = True)

,proportion
자동차,
일반세단,0.506167
컴팩트카,0.207000
SUV,0.194667
스포츠카,0.054333
고급차,0.037833


In [103]:
#[문제 21]'자동차' 범주를 인코딩 해보기
# 고급차,스포츠카를 1로. 나머지는 0

data_choice['자동차'] = np.where((data_choice['자동차'] == '고급차')| (data_choice['자동차'] == '스포츠카') , 1, 0)

<ipython-input-103-23fc8fc76b4a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_choice['자동차'] = np.where((data_choice['자동차'] == '고급차')| (data_choice['자동차'] == '스포츠카') , 1, 0)


In [104]:
#[문제 22] '자동차'가 변환된 것을 확인해보기
data_choice['자동차'].value_counts(normalize = True)


,proportion
자동차,
0,0.907833
1,0.092167


<font color="1E90FF"><b> 범주형(Object) -> 수치형으로 변경 완료!</span>

# **2. 스케일링**
* 컬럼별 수치의 단위가 다른 것을 확인 하였고, 컬럼별 범위를 조정하는 것이 필요한 상황이다.
* 범위를 일원화 시키기 위해서는 스케일링이 필요로 하다.
    * 종류는 'min-max scaling' 과 'standard scaling' 이 있는데 우리는 'standard scaling'을 통해서 변환 해보자.

In [105]:
#[문제 23] standard-scaling을 해보자.

# 1. standard-scaler import!(sklearn의 processing 활용)
from sklearn.preprocessing import StandardScaler

# 2. scaler라는 변수에  StandardScaler 넣어주기
scaler = StandardScaler()

# 3. 'data_choice'을  fit_transform 하여 'data_sc'로 저장
# 단, 데이터프레임을 스케일링하면, 결과가 넘파이 어레이로 나온다.
# 그래서 데이터프레임으로 다시 변환할 필요가 있다.(이때 칼럼 이름 필요)
data_sc = scaler.fit_transform(data_choice)
data_sc = pd.DataFrame(data_sc, columns=data_choice.columns)


In [106]:
# [문제 24] 스케일링이 잘 되었는지 'data_sc' 데이터를 확인해보자

print(data_sc.mean())        # 평균 ≈ 0
print(data_sc.std())         # 표준편차 ≈ 1

AGE                       -1.486219e-16
고용상태                       0.000000e+00
Willingness to pay/Stay   -6.276461e-17
상품타입                       3.671137e-17
교육수준                      -7.046215e-17
소득                        -2.605323e-17
월 납입액                      6.039613e-17
타 상품 보유 현황                -1.243450e-16
총지불금액                      1.119105e-16
거주지사이즈                     9.769963e-18
자동차                       -2.368476e-17
dtype: float64
AGE                        1.000042
고용상태                       0.000000
Willingness to pay/Stay    1.000042
상품타입                       1.000042
교육수준                       1.000042
소득                         1.000042
월 납입액                      1.000042
타 상품 보유 현황                 1.000042
총지불금액                      1.000042
거주지사이즈                     1.000042
자동차                        1.000042
dtype: float64


In [107]:
#[문제 25] 데이터 내보내기
# 2일차에 모델링에 활용하기 위해 내보내기를 해보자.
# data_sc 를 data_sc.csv 파일로 저장(to_csv 활용)
# index=False 파라미터를 설정해 줘야지 unnamed:0번 추가 인덱스가 안생기기에 꼭! 넣어서 저장!
data_sc.to_csv(path +'data_sc.csv', index=False)


In [108]:
#[문제 26] 데이터가 잘 들어갔는지 read_csv 활용해서 'data_sc.csv' 확인

data_check = pd.read_csv(path + 'data_sc.csv')

data_check.head()

,AGE,고용상태,Willingness to pay/Stay,상품타입,교육수준,소득,월 납입액,타 상품 보유 현황,총지불금액,거주지사이즈,자동차
0,-0.070427,0.0,-0.810943,-0.313685,-0.366062,-1.242413,-0.472671,-1.078127,0.265939,-0.340235,-0.318628
1,-1.853401,0.0,-0.325638,-0.313685,-0.366062,1.511484,-0.472671,1.414331,-1.342270,-0.340235,-0.318628
2,-0.961914,0.0,-0.504656,-0.313685,-0.366062,-0.649289,-0.943792,1.414331,-0.483815,-0.340235,-0.318628
3,1.712546,0.0,0.600217,3.187913,-0.366062,0.137715,0.234009,-0.247307,2.030230,2.939142,-0.318628
4,0.821059,0.0,0.119108,-0.313685,2.731780,-0.524540,0.469570,0.583512,1.146690,-0.340235,-0.318628


# **3. Mission 2-데이터 전처리 결론**

* 군집화 모델링을 하기 위해 범주형,수치형별 데이터 전처리가 필요했고 전처리 진행 완료
* 범주형은 수치형으로 변환을 하기 위해 인코딩 과정을 진행
* 단위가 각각 다른 수치형을 스케일링을 통해 모델 적용을 위한 기본 셋팅으로 변환

<font color="1E90FF"><b>→ 모델을 돌리기 전 데이터에 대한 이해도를 바탕으로 데이터의 전처리 진행(스케일링)<b><br>  

# <b>Mission 2 완료!
수고하셨습니다!